<a href="https://colab.research.google.com/github/YoheiFukuhara/recommender-system/blob/main/02_preprocess_shrink.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import Counter

import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
BASE_PATH = '/content/drive/MyDrive/ColabNotebooks/ML/Recommend/output/'
df = pd.read_csv(BASE_PATH+'edited_rating.csv')

In [ ]:
pd.options.display.float_format = '{:.2f}'.format
df.info()
display(df.describe())
display(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   movie_idx  int64  
dtypes: float64(1), int64(3)
memory usage: 610.4 MB


,userId,movieId,rating,movie_idx
count,20000263.00,20000263.00,20000263.00,20000263.00
mean,69044.87,9041.57,3.53,3612.60
std,40038.63,19789.48,1.05,4154.44
min,0.00,1.00,0.50,0.00
25%,34394.00,902.00,3.00,887.00
50%,69140.00,2167.00,3.50,2085.00
75%,103636.00,4770.00,4.00,4677.00
max,138492.00,131262.00,5.00,26743.00


,userId,movieId,rating,movie_idx
0,0,2,3.50,2
1,0,29,3.50,29
2,0,32,3.50,32
3,0,47,3.50,47
4,0,50,3.50,50
...,...,...,...,...
20000258,138492,68954,4.50,13821
20000259,138492,69526,4.50,13929
20000260,138492,69644,3.00,13942
20000261,138492,70286,5.00,14060


In [ ]:
print(f'unique number of user id: {df.userId.nunique()}')
print(f'unique number of movie id: {df.movie_idx.nunique()}')

unique number of user id: 138493
unique number of movie id: 26744


In [ ]:
%%time
# number of users and movies to keep
user_ids = [id for id, _ in Counter(df.userId).most_common(4000)] # the original is 10000
movie_ids = [id for id, _ in Counter(df.movie_idx).most_common(1000)] # the original is 2000

CPU times: user 9.83 s, sys: 43.4 ms, total: 9.87 s
Wall time: 9.82 s


In [ ]:
%%time
df_small = df[df.userId.isin(user_ids) & df.movie_idx.isin(movie_ids)].copy()
df_small.info()
display(df_small.describe())
display(df_small)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1982718 entries, 5400 to 19993378
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   movie_idx  int64  
dtypes: float64(1), int64(3)
memory usage: 75.6 MB


,userId,movieId,rating,movie_idx
count,1982718.00,1982718.00,1982718.00,1982718.00
mean,68624.56,5116.48,3.47,2799.43
std,39799.66,11798.80,1.00,2901.13
min,53.00,1.00,0.50,1.00
25%,34349.00,1029.00,3.00,1012.00
50%,69039.00,1997.00,3.50,1915.00
75%,102372.00,3578.00,4.00,3489.00
max,138436.00,81845.00,5.00,16233.00


,userId,movieId,rating,movie_idx
5400,53,1,4.00,1
5401,53,2,3.00,2
5402,53,5,3.00,5
5403,53,6,3.00,6
5404,53,10,4.00,10
...,...,...,...,...
19993363,138436,74458,3.50,14917
19993369,138436,78499,4.00,15448
19993372,138436,79132,4.50,15580
19993375,138436,80463,4.00,15901


CPU times: user 4.59 s, sys: 1.07 s, total: 5.65 s
Wall time: 5.63 s


In [ ]:
def create_old_map(ids):
    id_map = {}
    for i, old in enumerate(ids):
        id_map[old] = i
    print(len(id_map))
    return id_map

new_user_id_map = create_old_map(user_ids)
new_movie_id_map = create_old_map(movie_ids)

4000
1000


In [ ]:
%%time
df_small.loc[:,'userId'] = df_small.userId.map(lambda x: new_user_id_map[x])
df_small.loc[:,'movie_idx'] = df_small.movie_idx.map(lambda x: new_movie_id_map[x])

CPU times: user 1.66 s, sys: 132 ms, total: 1.79 s
Wall time: 1.78 s


In [ ]:
display(df_small.describe())

,userId,movieId,rating,movie_idx
count,1982718.00,1982718.00,1982718.00,1982718.00
mean,1796.93,5116.48,3.47,417.32
std,1164.18,11798.80,1.00,287.50
min,0.00,1.00,0.50,0.00
25%,765.00,1029.00,3.00,162.00
50%,1703.00,1997.00,3.50,379.00
75%,2776.00,3578.00,4.00,651.00
max,3999.00,81845.00,5.00,999.00


In [ ]:
df_small.to_csv(BASE_PATH+'very_small_rating.csv', index=False)